## 01b - Segment Songs with TweetyNet

```
C:\You> conda create -n vak-env python==3.9
C:\You> conda activate vak-env
(vak-env) C:\You> pip install torch===1.10.2 torchvision===0.11.3 -f https://download.pytorch.org/whl/torch_stable.html
(vak-env) C:\You> pip install vak==0.8.0
(vak-env) C:\You> pip install tweetynet


In [2]:
import numpy as np
import glob
import pandas as pd
import vak 
from pathlib import Path
import toml
from datetime import datetime
import os

In [15]:
vak.__version__

'0.8.0'

## Prep Spectrograms for Training

In [3]:
spect_params = {"fft_size": 512, 
                "step_size": 64, 
                "freq_cutoffs": [500, 10000], 
                "thresh": 6.25, 
                "transform_type" : "log_spect",  
                "freqbins_key" : 'f', 
                "timebins_key" : 't', 
                "spect_key" : 's', 
                "audio_path_key" : 'audio_path'}
labelset = ["n", "s"]
output_dir = Path("E:\\Final_Bird_Dataset\\2024_tweetynet_seg\\train\\")

In [4]:
import logging

logger = logging.getLogger('vak')

In [5]:
vak_df, csv_path = vak.core.prep(data_dir = "E:\\Final_Bird_Dataset\\FP1_project_birds\\segmented_songs\\", 
                                 purpose = "train", 
                                 output_dir = output_dir, 
                                 audio_format = "wav", 
                                 spect_params = spect_params, 
                                 annot_format = "notmat", 
                                 labelset = labelset,
                                 train_dur = 50, 
                                 val_dur = 15, 
                                 test_dur = 30, )


[########################################] | 100% Completed | 70.61 s
[########################################] | 100% Completed | 18.95 s
[########################################] | 100% Completed | 15.66 s


## Bird-Fold Cross Validation 

In [9]:
Training_Birds = ["B145", "B236", "B258", "B385", "B402", "B447", 
             "B507", "G255", "G397", "G402", "G413", "G437", 
             "G439", "G524", "G528", "O144", "O254", "O421", 
             "O440", "O512", "R402", "R425", "R469", "S132", 
             "S421", "S525", "S528", "Y389", "Y397", "Y425", 
             "Y440", "B524", "O434", "Y433", "Y453"] 
#the number of training birds should be 35
len(Training_Birds)

35

In [10]:
model_config_map = {'TweetyNet': {'optimizer': {'lr': 0.001},
  'network': {},
  'loss': {},
  'metrics': {}}}

In [14]:
for Bird in Training_Birds:
    sub_df = vak_df[~vak_df['audio_path'].str.contains(Bird)]
    csv_path = Path("E:\\Final_Bird_Dataset\\2024_tweetynet_seg\\train\\songs_prep_not_" + Bird + ".csv")
    sub_df.to_csv(csv_path)
    
    vak.core.train(model_config_map = model_config_map, 
                   csv_path = csv_path, 
                   labelset = set(labelset), 
                   window_size = 88, 
                   batch_size = 8, 
                   num_epochs = 2, 
                   num_workers = 2, 
                   root_results_dir = Path("E:\\Final_Bird_Dataset\\2024_tweetynet_seg\\results"), 
                   val_step = 50, 
                   ckpt_step = 200, 
                   patience = 4)
    
    song_path = Path("E:\\Final_Bird_Dataset\\FP1_project_birds\\segmented_songs\\" + Bird)
    
    if not os.path.exists(Path("E:\\Final_Bird_Dataset\\2024_tweetynet_seg\\segmented_songs_annotations\\" + Bird)):
        os.makedirs(Path("E:\\Final_Bird_Dataset\\2024_tweetynet_seg\\segmented_songs_annotations\\" + Bird))
    
    vak_df_predict, csv_path = vak.core.prep(data_dir = song_path, 
                                     purpose = 'predict', 
                                     output_dir = Path("E:\\Final_Bird_Dataset\\2024_tweetynet_seg\\segmented_songs_annotations\\" + Bird), 
                                     audio_format = "wav", 
                                     spect_params = spect_params,)
    
    most_recent_results = max(glob.glob('E:\\Final_Bird_Dataset\\2024_tweetynet_seg\\results/*/'), key = os.path.getmtime)
    checkpoint_path = Path(most_recent_results + "TweetyNet/checkpoints/max-val-acc-checkpoint.pt")
    labelmap_path = Path(most_recent_results + "labelmap.json")
    spect_scaler_path = Path(most_recent_results + "/StandardizeSpect")
    
    annot_csv_filename = Bird + ".annot.csv"
    output_dir = "E:\\Final_Bird_Dataset\\2024_tweetynet_seg\\segmented_songs_annotations\\"
    
    vak.core.predict(csv_path = csv_path, 
                     checkpoint_path = checkpoint_path, 
                     labelmap_path = labelmap_path, 
                     model_config_map = model_config_map, 
                     window_size = 88, 
                     num_workers = 4, 
                     spect_scaler_path = spect_scaler_path, 
                     device = "cpu", 
                     annot_csv_filename = annot_csv_filename, 
                     output_dir = output_dir,)

batch 3 / 4: 100%|██████████| 4/4 [00:04<00:00,  1.24s/it]         | 348/3861 [00:50<02:12, 26.53it/s]
Epoch 1, batch 349. Loss: 0.0618. Global step: 349:   9%|▉         | 348/3861 [00:55<09:18,  6.29it/s]


[########################################] | 100% Completed | 10.11 s
[########################################] | 100% Completed | 9.23 ss
[########################################] | 100% Completed | 9.23 ss


batch 3 / 4: 100%|██████████| 4/4 [00:05<00:00,  1.42s/it]         | 349/4356 [00:51<03:08, 21.24it/s]
Epoch 1, batch 349. Loss: 0.2804. Global step: 349:   8%|▊         | 349/4356 [00:57<11:02,  6.05it/s]


[########################################] | 100% Completed | 11.81 s
[########################################] | 100% Completed | 9.36 ss
[########################################] | 100% Completed | 9.37 ss


batch 3 / 4: 100%|██████████| 4/4 [00:05<00:00,  1.38s/it]         | 396/4356 [01:03<02:40, 24.60it/s]
Epoch 1, batch 399. Loss: 0.0843. Global step: 399:   9%|▉         | 396/4356 [01:08<11:26,  5.77it/s]


...

batch 3 / 4: 100%|██████████| 4/4 [00:04<00:00,  1.20s/it]▍        | 648/4356 [01:23<02:17, 26.89it/s]
Epoch 1, batch 649. Loss: 0.1523. Global step: 649:  15%|█▍        | 648/4356 [01:28<08:26,  7.32it/s]


[########################################] | 100% Completed | 13.01 s
[########################################] | 100% Completed | 10.63 s
[########################################] | 100% Completed | 10.37 s


batch 3 / 4: 100%|██████████| 4/4 [00:04<00:00,  1.19s/it]▌        | 696/4356 [01:29<02:14, 27.24it/s]
Epoch 1, batch 699. Loss: 0.3417. Global step: 699:  16%|█▌        | 696/4356 [01:34<08:17,  7.36it/s]


[########################################] | 100% Completed | 14.57 s
[########################################] | 100% Completed | 9.51 ss
[########################################] | 100% Completed | 9.09 ss


100%|██████████| 49/49 [00:07<00:00,  6.83it/s]
